In [24]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf

In [17]:
if isinstance(checkpoint, dict):
    print("가중치만 포함")
    for layer_name in checkpoint.keys():
        print(f"Layer: {layer_name}") # 레이어 출력
else:
    print("구조, 가중치 포함")
    print("모델 객체:", checkpoint)

가중치만 포함
Layer: epoch
Layer: model_state_dict
Layer: optimizer_state_dict
Layer: min_train_loss
Layer: min_valid_loss


## ResNet-50 신경망을 사용 하는듯

In [18]:
model = models.resnet50(num_classes=3)

In [21]:
model.load_state_dict(torch.load('openfieldModel.pt', map_location=torch.device('cpu'), weights_only=True)['model_state_dict'])

<All keys matched successfully>

In [22]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## 모델 테스트

In [36]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [33]:
from PIL import Image

# 이미지 로드
image = Image.open('test_img/V006_79_0_00_01_01_13_0_a01_20201102_0000_S01_1.jpg').convert('RGB')  # 이미지를 RGB로 변환
input_tensor = transform(image)  # 전처리 적용
input_batch = input_tensor.unsqueeze(0)  # 배치 차원 추가
print(input_batch.size())

torch.Size([1, 3, 224, 224])


In [34]:
with torch.no_grad():  # 경량화 모드로 전환
    output = model(input_batch)
    # 예측 결과를 클래스 인덱스로 변환
    _, predicted = torch.max(output, 1)

print(f'Predicted class index: {predicted.item()}')  # 0, 1, 2 중 하나의 결과 출력

Predicted class index: 2


In [30]:
import torch
from torchvision import transforms
from PIL import Image

# 예시: 이미지를 전처리하여 저장한 텐서라고 가정합니다.
# 여기서 input_tensor는 [batch_size, channels, height, width] 형태의 텐서입니다.

def tensor_to_image(tensor):
    # 텐서를 [batch_size, C, H, W]에서 [C, H, W]로 변환
    tensor = tensor.squeeze(0)  # 배치 차원 제거
    tensor = tensor.permute(1, 2, 0)  # 차원 재배치 [C, H, W] -> [H, W, C]
    tensor = tensor.clamp(0, 1)  # 값 범위를 [0, 1]로 제한
    tensor = (tensor * 255).byte()  # [0, 1] 범위를 [0, 255]로 변환하여 byte 타입으로 변경
    return Image.fromarray(tensor.numpy())  # NumPy 배열로 변환 후 PIL 이미지로 변환

# 텐서를 이미지로 변환
image = tensor_to_image(input_tensor)

# 이미지 저장
image.save('output_image.jpg')  # 원하는 경로와 파일명으로 저장